<a href="https://colab.research.google.com/github/kleineidam/LeafTransport/blob/main/ant_detection_analysis01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.9 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralyti

MessageError: Error: credential propagation was unsuccessful

In [1]:

!pip install ultralytics --quiet

from google.colab import drive
import os
import cv2
import pandas as pd
from pathlib import Path
from ultralytics import YOLO

# Mount Google Drive
# If prompted, authorize Colab to access your Google Drive.
# Attempt to unmount drive first, in case of previous failed attempts
try:
    drive.flush_and_unmount()
    print("✅ Google Drive unmounted successfully (if previously mounted).")
except ValueError:
    print("ℹ️ Google Drive was not mounted, proceeding to mount.")

drive.mount('/content/drive')
# Check if mount was successful (optional but good practice)
if os.path.exists('/content/drive/MyDrive'):
    print("✅ Google Drive mounted successfully.")
else:
    print("❌ Google Drive mount failed. Please check permissions and try again.")

# Check if mount was successful (optional but good practice)
if os.path.exists('/content/drive/MyDrive'):
    print("✅ Google Drive mounted successfully.")
else:
    print("❌ Google Drive mount failed. Please check permissions and try again.")

BASE_DIR = Path('/content/drive/MyDrive/AntCountingProject').resolve()
print(f"Base directory set to: {BASE_DIR}")

MODEL_PATH = BASE_DIR / 'best.pt'
IMAGE_DIR = BASE_DIR / 'input_images'
RESULTS_CSV = BASE_DIR / 'ant_counts.csv'

ROIS = [
    (55, 309, 1891, 763),    # ROI 1: density
    (361, 169, 653, 459),    # ROI 2: testing quadrant oben
    (361, 599, 653, 900)     # ROI 3: testing quadrant unten
]

# Verify files and directories
if not MODEL_PATH.exists():
    raise FileNotFoundError(f"Model file not found: {MODEL_PATH}")
if not IMAGE_DIR.exists():
    raise FileNotFoundError(f"Image input folder not found: {IMAGE_DIR}")

# Load YOLOv8 model
# Ensure that the drive is mounted and MODEL_PATH exists before this line.
model = YOLO(str(MODEL_PATH))

def count_ants_in_rois(detections, rois):
    counts = [0] * len(rois)
    # Access detections.boxes.xyxy safely, it should be available if the model predicted
    if detections.boxes is not None:
        for box in detections.boxes.xyxy:
            x1, y1, x2, y2 = map(int, box[:4])
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            for i, (rx1, ry1, rx2, ry2) in enumerate(rois):
                if rx1 <= cx <= rx2 and ry1 <= cy <= ry2:
                    counts[i] += 1
    return counts

data = []
# Check if the image directory is accessible and list files
if IMAGE_DIR.is_dir():
    for image_file in os.listdir(IMAGE_DIR):
        if image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = IMAGE_DIR / image_file
            img = cv2.imread(str(image_path))
            if img is None:
                print(f"⚠️ Warning: Could not read image {image_file}. Skipping.")
                continue

            results = model(img)
            # results is a list of Results objects, usually just one for a single image input
            if results:
                detections = results[0]
                counts = count_ants_in_rois(detections, ROIS)

                data.append([image_file, *counts])
            else:
                 print(f"⚠️ Warning: No results returned for image {image_file}. Skipping.")
else:
    print(f"❌ Image input folder not found or not accessible: {IMAGE_DIR}")


# Only create DataFrame if data is not empty
if data:
    df = pd.DataFrame(data, columns=['Filename', 'Region 1', 'Region 2', 'Region 3'])
    df.to_csv(RESULTS_CSV, index=False)
    print(f"✅ Results saved to {RESULTS_CSV}")
else:
    print("ℹ️ No images processed or no ants detected. No results file created.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.6 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralyti

MessageError: Error: credential propagation was unsuccessful